# Capítulo 06 — Fine-Tuning: A Especialização do Modelo

Neste capítulo, vamos realizar uma "cirurgia neural". Pegaremos o GPTMini que aprendeu a ler e escrever no Capítulo 05 e o ensinaremos a classificar mensagens como **Normal** ou **Spam**.

--- 
### 🎯 O Poder da Especialização
O Fine-tuning não apaga o que o modelo sabe; ele apenas direciona esse conhecimento para uma tarefa específica.

![Pretrain vs Finetune](./infograficos/01-pretrain-vs-finetune.png)

In [ ]:
# ============================================================
# Setup do repositório no Colab
# ============================================================
import os, sys
REPO_NAME = "fazendo-um-llm-do-zero"
if 'google.colab' in str(get_ipython()):
    if not os.path.exists(REPO_NAME):
        os.system(f"git clone https://github.com/vongrossi/{REPO_NAME}.git")
    if os.path.exists(REPO_NAME) and os.getcwd().split('/')[-1] != REPO_NAME:
        os.chdir(REPO_NAME)
if os.getcwd() not in sys.path: sys.path.append(os.getcwd())
print("📂 Diretório atual:", os.getcwd())

In [ ]:
# ============================================================
# 1. Setup e Conexão com a Inteligência Base
# ============================================================
import os, sys, torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from lib.gptmini import GPTConfig, GPTMini

device = "cuda" if torch.cuda.is_available() else "cpu"

if not os.path.exists("gpt_checkpoint.pt"):
    from google.colab import files
    print("📤 Por favor, suba o 'gpt_checkpoint.pt' gerado no Capítulo 05:")
    uploaded = files.upload()

try:
    ckpt = torch.load("gpt_checkpoint.pt", map_location=device, weights_only=False)
    stoi, itos = ckpt['stoi'], ckpt['itos']
    vocab_size = len(stoi)
    print(f"✅ Inteligência Base Carregada! Vocabulário: {vocab_size} caracteres.")
except Exception as e:
    print(f"❌ ERRO AO CARREGAR: {e}")

## 2. Preparando os Dados de Missão

Precisamos de exemplos de SPAM para que o modelo entenda o padrão de mensagens maliciosas.

In [ ]:
raw_data = [
    ("ganhe 1 milhao agora clique aqui", 1), # Spam
    ("oferta imperdivel premio gratis", 1),   # Spam
    ("seu premio esta esperando resgate", 1), # Spam
    ("ola tudo bem como voce esta", 0),      # Normal
    ("reuniao de equipe amanha as dez", 0),   # Normal
    ("voce vai no churrasco no domingo", 0)   # Normal
]

encode = lambda s: [stoi[c] for c in s.lower() if c in stoi]

def build_dataset(data, max_len=32):
    X, Y = [], []
    for text, label in data:
        ids = encode(text)
        ids = ids[:max_len] + [stoi.get(' ', 0)] * (max_len - len(ids))
        X.append(ids); Y.append(label)
    return torch.tensor(X).to(device), torch.tensor(Y).to(device)

X_train, Y_train = build_dataset(raw_data)
print(f"📊 Dataset Processado: {len(X_train)} exemplos prontos.")

## 3. Criando o Classificador

Acoplamos a "Cabeça de Classificação" ao Backbone do Transformer.

![Classification Head](./infograficos/02-classification-head.png)

In [ ]:
class GPTClassifier(nn.Module):
    def __init__(self, backbone, num_classes=2):
        super().__init__(); self.backbone = backbone
        self.clf_head = nn.Linear(backbone.config.d_model, num_classes)
    def forward(self, x):
        x = self.backbone.emb(x)
        x = self.backbone.blocks(x)
        x = self.backbone.ln_f(x)
        return self.clf_head(x[:, -1, :])

backbone = GPTMini(ckpt['config']).to(device)
backbone.load_state_dict(ckpt['state_dict'])
model = GPTClassifier(backbone).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-4)
print("🏗️ Modelo especializado pronto.")

## 4. O Treinamento do Especialista

O modelo aprende a separar as classes Spam e Normal.

In [ ]:
loss_history = []
model.train()
for step in range(201):
    logits = model(X_train)
    loss = F.cross_entropy(logits, Y_train)
    optimizer.zero_grad(); loss.backward(); optimizer.step()
    loss_history.append(loss.item())
    if step % 50 == 0:
        acc = (torch.argmax(logits, dim=-1) == Y_train).float().mean()
        print(f"Passo {step:03d} | Erro: {loss.item():.4f} | Acurácia: {acc*100:.1f}%")

plt.figure(figsize=(8, 3))
plt.plot(loss_history, color='#34A853')
plt.title("Curva de Especialização")
plt.show()

## 5. Teste de Campo

Vamos testar com frases inéditas.

In [ ]:
def classify(text):
    model.eval()
    with torch.no_grad():
        ids = encode(text)
        ids_tensor = torch.tensor(ids[:32] + [stoi.get(' ', 0)] * (32 - len(ids))).unsqueeze(0).to(device)
        probs = F.softmax(model(ids_tensor), dim=-1)
        pred = torch.argmax(probs, dim=-1).item()
        label = "🚨 SPAM" if pred == 1 else "✅ NORMAL"
        return f"{label} ({probs[0, pred].item()*100:.1f}% de confiança)"

print(f"'ganhe premio agora': {classify('ganhe premio agora')}")
print(f"'ola tudo bem amigo': {classify('ola tudo bem amigo')}")